In [ ]:
from src import utils
import os
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
data_path = os.path.normpath(r'../data/raw/BL_pearson_organpair.csv')
raw_data = pd.read_csv(data_path, index_col=0)
# raw_data = pd.read_csv('../data/raw/pearson_organpair_4 strains_corrected.csv', index_col=0)
raw_data.head()



,bone_BL_24h_r1,bone_BL_24h_r2,bone_BL_24h_r3,bone_BL_24h_r4,bone_BL_3h_r1,bone_BL_48h_r1,bone_BL_48h_r2,bone_BL_48h_r3,bone_BL_48h_r4,bone_BL_48h_r5,...,spleen_BL_48h_r3,spleen_BL_48h_r4,spleen_BL_r1,spleen_BL_r2,spleen_BL_r3,spleen_BL_r4,testis_BL_8h_r1,testis_BL_8h_r2,testis_BL_r1,testis_BL_r2
bone_BL_24h_r1,1.000000,0.593909,0.557183,0.672997,0.557368,0.751965,0.785804,0.611671,0.815540,0.801837,...,0.148545,0.138057,0.108920,0.588349,0.058028,0.114238,0.097689,0.054613,0.097397,0.096090
bone_BL_24h_r2,0.593909,1.000000,0.903332,0.887138,0.926070,0.874727,0.894454,0.926441,0.801978,0.782366,...,0.103078,0.080471,0.239363,0.328431,0.205129,0.250387,0.083549,0.061643,0.103562,0.088354
bone_BL_24h_r3,0.557183,0.903332,1.000000,0.973454,0.880818,0.915612,0.895245,0.965719,0.849069,0.745841,...,0.070438,0.067942,0.045224,0.271326,0.024706,0.047574,0.021556,0.020687,0.032589,0.027502
bone_BL_24h_r4,0.672997,0.887138,0.973454,1.000000,0.877562,0.949721,0.938727,0.947019,0.909073,0.823679,...,0.062770,0.063825,0.040597,0.372586,0.022371,0.041669,0.026505,0.020306,0.037196,0.034492
bone_BL_3h_r1,0.557368,0.926070,0.880818,0.877562,1.000000,0.837083,0.858010,0.872859,0.764717,0.722455,...,0.166132,0.138480,0.287625,0.353987,0.218988,0.308186,0.134524,0.117724,0.175454,0.155294


In [11]:
import pandas as pd
import numpy as np
import os
from itertools import combinations, combinations_with_replacement, product
import string
import matplotlib.pyplot as plt


def fix_time_label(data):
    # TODO
    return complete_data_label(data)

def check_data_cols(data):
    
    cols = data.columns.values
    index = data.index.values
    
    print(np.all(cols == index))

def complete_data_label(data):

    cols = data.columns.values
    complete_labels = [complete_label(item) for item in cols]
    complete_labels

    data.index = complete_labels
    data.columns = complete_labels
    
    return data

def get_same_row_col_labels(data):
    
    cols = set(data.columns)
    rows = set(data.index)
    row_col_labels = sorted(list(cols.intersection(rows)))
    
    return data.loc[row_col_labels, row_col_labels]

def get_cell_diagonal_labels(n):
    
    numbers = np.arange(2, n+2)
    letters = list(string.ascii_uppercase)[1:n+1]
    
    labels = [letters[i] + str(numbers[i]) for i in range(n)]
    
    return labels

def complete_label(label):
    
    label_comps = label.split("_")
    
    if len(label_comps) == 3:
        label_comps.insert(2, '0h')

    new_label = '_'.join(label_comps)
    
    return new_label

def get_var(label, idx):
    
    label_comps = label.split('_')
    
    return label_comps[idx]

def get_var_row_subset_df(var_name, var_idx, data):
    
    is_var_row = [(var_name == get_var(name, var_idx)) for name in data.index]
    
    return data.loc[is_var_row, :].copy()

def get_var_pair_df(var_pair, data, exclude=False):
    
    
    cols = data.columns
    rows = data.index
    
    cols_comps = [item.split('_') for item in cols]
    
    if exclude:
        is_var_2 = [(var_pair[1] not in item.split('_'))  for item in cols]
        is_var_1 = [(var_pair[0] not in item.split('_'))  for item in rows]
    else:
        is_var_2 = [(var_pair[1] in item.split('_'))  for item in cols]
        is_var_1 = [(var_pair[0] in item.split('_'))  for item in rows]
    
    
    var_pair_subset = data.loc[is_var_1, is_var_2]    
    
    return var_pair_subset

def get_var_list(var_idx, data):
    
    var_list = sorted(set([get_var(name, var_idx) for name in data.columns]))
    
    return var_list

def get_var_pair(var_idx, data, include_permutation=False):
    
    var_list = sorted(set([get_var(name, var_idx) for name in data.columns]))
#     var_pairs = list(product(var_list, repeat=2))
    if include_permutation:
        var_pairs = list(product(var_list, repeat=2))
    else:
        var_pairs = list(combinations(var_list, 2))

        for item in var_list:
            self_self = item, item
            var_pairs.append(self_self)
    
    return var_pairs
    

# def get_common_var_2(var_1_idx, var_2_idx, data ):
    
#     var_1_list = get_var_list(var_1_idx, data)
    
#     var_2s_for_var_1 = {}
#     for var_1 in var_1_list:

#         var_1_var_2s = set([get_var(name, var_2_idx)  for name in data.columns if var_1 in name])
#         var_2s_for_var_1[var_1] = var_1_var_2s
    
#     common_var_2s = list(set.intersection(*list(var_2s_for_var_1.values())))
    
#     return common_var_2s

def get_var_commons(ref_var_idx, common_var_idx, data ):
    
    ref_var_list = get_var_list(ref_var_idx, data)
    
    ref_vars_vars = {}
    for ref_var in ref_var_list:

        ref_var_vars = set([get_var(col_name, common_var_idx)  for col_name in data.columns if ref_var in col_name])
        ref_vars_vars[ref_var] = ref_var_vars
    
    common_vars = list(set.intersection(*list(ref_vars_vars.values())))
    
    return common_vars


def filter_var_subset_df(var_list, var_idx, data):
    
    cols = data.columns
    
    has_var_list_label =  [get_var(name, var_idx) in var_list for name in cols]
    
    df_var_list_subset = data.loc[has_var_list_label, has_var_list_label]
    
    return df_var_list_subset
    

def get_likelihood(x, mu=0, sigma=1):
    var = sigma **2
    L = (1 / (np.sqrt(2 * np.pi * var))) * \
    np.exp(-(x - mu)**2 / (2*var))
    
    return L

def geo_mean(iterable):
    a = np.array(iterable)
    return (a**(1.0/len(a))).prod()

def get_strain_var_likelihood(var_str_idx, var_2_idx, data,  var2_pair=None, geom_normalize=True, use_common_var_2=False):
    
    df = data.copy()
    
    # filter df so as to have only organs common across all strains
    if use_common_var_2 is True:
        common_var_2s = get_var_commons(ref_var_idx=var_str_idx, common_var_idx=var_2_idx, data=df)
        df = filter_var_subset_df(var_list=common_var_2s, var_idx=var_2_idx, data=df).copy()
    
    # check if there is a variable 2 to filter 
    if var2_pair is not None:
        df = get_var_pair_df(var2_pair, df)

    var_1_pairs = get_var_pair(var_idx=var_str_idx, data=df, include_permutation=True)
#     print(var_1_pairs)
    var_2_pairs_combination = get_var_pair(var_idx=var_2_idx, data=df, include_permutation=False)
    var_2_pairs_permuted = get_var_pair(var_idx=var_2_idx, data=df, include_permutation=True)
#     print(var_2_pairs)
    
    var_1_list = get_var_list(var_idx=var_str_idx, data=df)

    var_var_df = pd.DataFrame(index=var_1_list, columns=var_1_list)
    

    for i, var_1_pair in enumerate(var_1_pairs):
        
        corr_df_var_1_x = get_var_pair_df((var_1_pair[0], var_1_pair[0]), data=df)
        corr_df_var_1_y = get_var_pair_df((var_1_pair[1], var_1_pair[1]), data=df)
#         corr_df_var_1_xy = get_var_pair_df((var_1_pair[0], var_1_pair[1]), data=df)

#         print(corr_df_var_1_x)
#         print(corr_df_var_1_y)

        if var_1_pair[0] != var_1_pair[1]:
            var_2_pairs = var_2_pairs_permuted
        else:
            var_2_pairs = var_2_pairs_combination
        
        var_2_lhs = []
        for j, var_2_pair in enumerate(var_2_pairs):
            
#             corr_df_var_1_xy_var_2_x = get_var_pair_df((var_2_pair[0], var_2_pair[0]), data=corr_df_var_1_xy).values
#             corr_df_var_1_xy_var_2_y = get_var_pair_df((var_2_pair[1], var_2_pair[1]), data=corr_df_var_1_xy).values

            corr_df_var_1_x_var_2 = get_var_pair_df((var_2_pair[0], var_2_pair[1]), data=corr_df_var_1_x).values
            corr_df_var_1_y_var_2 = get_var_pair_df((var_2_pair[0], var_2_pair[1]), data=corr_df_var_1_y).values
            
            # get rid of diagonals ( will typically have ones)
#             corr_df_var_1_x_var_2 = corr_df_var_1_x_var_2[corr_df_var_1_x_var_2!=1]
#             corr_df_var_1_y_var_2 = corr_df_var_1_y_var_2[corr_df_var_1_y_var_2!=1]

            # take upper diagonal if same var_2 (e.g same organs)
            if var_2_pair[0] == var_2_pair[1]:
                corr_df_var_1_x_var_2 = corr_df_var_1_x_var_2[np.tril_indices_from(corr_df_var_1_x_var_2)]
                corr_df_var_1_y_var_2 = corr_df_var_1_y_var_2[np.tril_indices_from(corr_df_var_1_y_var_2)]
            

            if (corr_df_var_1_x_var_2.size >= 2) and (corr_df_var_1_y_var_2.size >= 2):

                # row is reference, row is mean
                mu = corr_df_var_1_x_var_2.mean()
                sigma = corr_df_var_1_x_var_2.std(ddof=1)  # sigma can be zero if all values are same
                x = corr_df_var_1_y_var_2
                
                var_1_pair_to_check = (('AJ', 'BL'), ('BL', 'BL')); var_2_pair_to_check = (('bone', 'brain'), ('bone', 'bone'))
                
                if var_1_pair in var_1_pair_to_check and var_2_pair in var_2_pair_to_check:
                    print (f"mu for {var_1_pair} and {var_2_pair} = {mu} and std = {sigma}")
                    
                if sigma != 0:
                    var_2_L = get_likelihood(x, mu, sigma)
                    
                
                    if geom_normalize == True:
                # find the n_root to keep likelihood computation uniform for different n_L
                        var_2_L = var_2_L ** (1/var_2_L.size)

                    var_2_L = var_2_L.prod()
                    var_2_lhs.append(var_2_L)
    
            
        var_2_lhs = np.array(var_2_lhs)
        var_2_lhs = var_2_lhs[~np.isnan(var_2_lhs)]
#         print(var_2_lhs)

        lh = geo_mean(var_2_lhs)
        var_var_df.loc[var_1_pair[0], var_1_pair[1]] = lh
        
    return var_var_df

def get_organ_var_likelihood(var_str_idx, var_2_idx, data,  var2_pair=None, geom_normalize=True, use_common_var_2=False):
    
    df = data.copy()
    
    # filter df so as to have only organs common across all strains
    if use_common_var_2 is True:
        common_var_2s = get_var_commons(ref_var_idx=var_2_idx, common_var_idx=var_str_idx, data=df)
        df = filter_var_subset_df(var_list=common_var_2s, var_idx=var_str_idx, data=df).copy()
        print(common_var_2s)

    # check if there is a variable 2 to filter 
    if var2_pair is not None:
        df = get_var_pair_df(var2_pair, df)

    var_1_pairs = get_var_pair(var_idx=var_str_idx, data=df, include_permutation=True)
    var_2_pairs_combination = get_var_pair(var_idx=var_2_idx, data=df, include_permutation=False)
    var_2_pairs_permuted = get_var_pair(var_idx=var_2_idx, data=df, include_permutation=True)
    
    var_1_list = get_var_list(var_idx=var_str_idx, data=df)
#     var_2_list = get_var_list(var_idx=var_2_idx, data=df)

    var_var_df = pd.DataFrame(index=var_1_list, columns=var_1_list)
    
    


    for i, var_1_pair in enumerate(var_1_pairs):
        
        corr_df_var_1_xy = get_var_pair_df((var_1_pair[0], var_1_pair[1]), data=df)
#         corr_df_var_1_y = get_var_pair_df((var_1_pair[1], var_1_pair[1]), data=df)
        
        if var_1_pair[0] != var_1_pair[1]:
            var_2_pairs = var_2_pairs_permuted
        else:
            var_2_pairs = var_2_pairs_combination
        
        var_2_lhs = []
        for j, var_2_pair in enumerate(var_2_pairs):

            corr_df_var_1_xy_var_2_x = get_var_pair_df((var_2_pair[0], var_2_pair[0]), data=corr_df_var_1_xy).values
            corr_df_var_1_xy_var_2_y = get_var_pair_df((var_2_pair[1], var_2_pair[1]), data=corr_df_var_1_xy).values
            
            # get rid of diagonals ( will typically have ones)
#             corr_df_var_1_xy_var_2_x = corr_df_var_1_xy_var_2_x[corr_df_var_1_xy_var_2_x!=1]
#             corr_df_var_1_xy_var_2_y = corr_df_var_1_xy_var_2_y[corr_df_var_1_xy_var_2_y!=1]

             # take upper diagonal if same organs
            if var_1_pair[0] == var_1_pair[1]:
#                 print(corr_df_var_1_xy_var_2_x)
                corr_df_var_1_xy_var_2_x = corr_df_var_1_xy_var_2_x[np.tril_indices_from(corr_df_var_1_xy_var_2_x)]
                corr_df_var_1_xy_var_2_y = corr_df_var_1_xy_var_2_y[np.tril_indices_from(corr_df_var_1_xy_var_2_y)]

            if (corr_df_var_1_xy_var_2_x.size >= 2) and (corr_df_var_1_xy_var_2_y.size >= 2):


                # row is reference, row is mean
                mu = corr_df_var_1_xy_var_2_x.mean()
                sigma = corr_df_var_1_xy_var_2_x.std(ddof=1)  # sigma can be zero if all values are same
                x = corr_df_var_1_xy_var_2_y

                var_1_pair_to_check = (('lung', 'kidney'), ('kidney', 'kidney')); var_2_pair_to_check = (('AJ', 'BL'), ('BL', 'BL'))
                
                if var_1_pair in var_1_pair_to_check and var_2_pair in var_2_pair_to_check:
                    print (f"mu for {var_1_pair} and {var_2_pair} = {mu} and std = {sigma}")
                
                if sigma != 0:
                
                    var_2_L = get_likelihood(x, mu, sigma)
                    if geom_normalize == True:
                # find the n_root to keep likelihood computation uniform for different n_L
                        var_2_L = var_2_L ** (1/var_2_L.size)

                    var_2_L = var_2_L.prod()
                    var_2_lhs.append(var_2_L)
    
            
        var_2_lhs = np.array(var_2_lhs)
        var_2_lhs = var_2_lhs[~np.isnan(var_2_lhs)]
        
        lh = geo_mean(var_2_lhs)
        var_var_df.loc[var_1_pair[0], var_1_pair[1]] = lh
        
    return var_var_df

def get_ranking(raw_df_path, save_path):
    
    data_hr_pair_mat = pd.ExcelFile(raw_df_path)

    with pd.ExcelWriter(os.path.join(save_path), engine='xlsxwriter') as writer:

        workbook = writer.book
        fail_format = workbook.add_format({'bg_color': '#FFC7CE',
                       'font_color': '#9C0006'})
        for i, sheet in enumerate(data_hr_pair_mat.sheet_names):
            sheet_df = data_hr_pair_mat.parse(sheet_name=sheet, index_col=0)
            #sheet_df = sheet_df.iloc[:, :-2]
            rank_df = sheet_df.rank(ascending=False, method='max', axis=0)
            #rank_df['similarity_rank'] = rank_df.mean(axis=1)
            rank_df.to_excel(writer, sheet_name=sheet)

            nrows = rank_df.shape[0]
            ncols = rank_df.shape[1]

            cell_labels = get_cell_diagonal_labels(nrows)

            worksheet = writer.sheets[sheet]
            for j, cell_label in enumerate(cell_labels):



                worksheet.conditional_format(cell_label, {'type':     'cell',
                                           'criteria': 'not equal to',
                                           'value':    1,
                                       'format':   fail_format})
    data_hr_pair_mat.close()
    

def get_organ_var_likelihood_debug(var_str_idx, var_2_idx, data,  var2_pair=None, geom_normalize=True, use_common_var_2=False):
    
    df = data.copy()
    
    # filter df so as to have only organs common across all strains
    if use_common_var_2 is True:
        common_var_2s = get_var_commons(ref_var_idx=var_2_idx, common_var_idx=var_str_idx, data=df)
        df = filter_var_subset_df(var_list=common_var_2s, var_idx=var_str_idx, data=df).copy()

    # check if there is a variable 2 to filter 
    if var2_pair is not None:
        df = get_var_pair_df(var2_pair, df)

    var_1_pairs = get_var_pair(var_idx=var_str_idx, data=df, include_permutation=False)
#     var_2_pairs_combination = get_var_pair(var_idx=var_2_idx, data=df, include_permutation=True)
    var_2_pairs_permuted = get_var_pair(var_idx=var_2_idx, data=df, include_permutation=True)
    
    var_1_list = get_var_list(var_idx=var_str_idx, data=df)
#     var_2_list = get_var_list(var_idx=var_2_idx, data=df)

    # dataframe of non-aggregated likelihood
    var_var_df = pd.DataFrame(index=var_1_list, columns=var_1_list)
    
    var_pairs_df = pd.DataFrame(index=var_2_pairs_permuted, columns=var_1_pairs)
    var_pairs_df.columns = var_pairs_df.columns.map('-'.join)
    var_pairs_df.index = var_pairs_df.index.map('-'.join)
    
    
    var_pairs_mean_df =  pd.DataFrame(index=var_pairs_df.index, columns=var_pairs_df.columns)


    for i, var_1_pair in enumerate(var_1_pairs):
        
        corr_df_var_1_xy = get_var_pair_df((var_1_pair[0], var_1_pair[1]), data=df)
#         corr_df_var_1_y = get_var_pair_df((var_1_pair[1], var_1_pair[1]), data=df)
        
#         if var_1_pair[0] != var_1_pair[1]:
#             var_2_pairs = var_2_pairs_permuted
#         else:
#             var_2_pairs = var_2_pairs_combination
        var_2_pairs = var_2_pairs_permuted
    
        var_2_lhs = []
        for j, var_2_pair in enumerate(var_2_pairs):
            
            

            corr_df_var_1_xy_var_2_x = get_var_pair_df((var_2_pair[0], var_2_pair[0]), data=corr_df_var_1_xy).values
            corr_df_var_1_xy_var_2_y = get_var_pair_df((var_2_pair[1], var_2_pair[1]), data=corr_df_var_1_xy).values
            
            # get rid of diagonals ( will typically have ones)
#             corr_df_var_1_xy_var_2_x = corr_df_var_1_xy_var_2_x[corr_df_var_1_xy_var_2_x!=1]
#             corr_df_var_1_xy_var_2_y = corr_df_var_1_xy_var_2_y[corr_df_var_1_xy_var_2_y!=1]

             # take upper diagonal if same organs
            if var_1_pair[0] == var_1_pair[1]:
#                 print(corr_df_var_1_xy_var_2_x)
                corr_df_var_1_xy_var_2_x = corr_df_var_1_xy_var_2_x[np.tril_indices_from(corr_df_var_1_xy_var_2_x)]
                corr_df_var_1_xy_var_2_y = corr_df_var_1_xy_var_2_y[np.tril_indices_from(corr_df_var_1_xy_var_2_y)]

            if (corr_df_var_1_xy_var_2_x.size >= 2) and (corr_df_var_1_xy_var_2_y.size >= 2):


                # row is reference, row is mean
                mu = corr_df_var_1_xy_var_2_x.mean()
                sigma = corr_df_var_1_xy_var_2_x.std(ddof=1)  # sigma can be zero if all values are same
                x = corr_df_var_1_xy_var_2_y

                var_1_pair_to_check = (('lung', 'kidney'), ('kidney', 'kidney')); var_2_pair_to_check = (('AJ', 'BL'), ('BL', 'BL'))
                
                if var_1_pair in var_1_pair_to_check and var_2_pair in var_2_pair_to_check:
                    print (f"mu for {var_1_pair} and {var_2_pair} = {mu} and std = {sigma}")
                
                if sigma != 0:
                
                    var_2_L = get_likelihood(x, mu, sigma)
                    if geom_normalize == True:
                # find the n_root to keep likelihood computation uniform for different n_L
                        var_2_L = var_2_L ** (1/var_2_L.size)
                    
                    var_2_L = var_2_L.prod()
                    var_2_lhs.append(var_2_L)
                    
                    
#                     if j ==1:
#                         print(var_2_pair)
#                     print(var_2_pair, var_1_pair)
                    var_pairs_df.loc['-'.join(var_2_pair), '-'.join(var_1_pair)] = var_2_L
                    var_pairs_mean_df.loc['-'.join(var_2_pair), '-'.join(var_1_pair)] = (mu, sigma)

                    
                    
    
            
        var_2_lhs = np.array(var_2_lhs)
        var_2_lhs = var_2_lhs[~np.isnan(var_2_lhs)]
        
        lh = geo_mean(var_2_lhs)
        if var_1_pair in [('bone', 'brain')]:
            print(var_2_lhs)
            print(lh)
        var_var_df.loc[var_1_pair[0], var_1_pair[1]] = lh
        
    print('== lh values== \n',var_pairs_df)
    print('== mean and std. values== \n',var_pairs_mean_df)
    return var_var_df

# Processing

In [13]:
clean_data = fix_time_label(raw_data)
# clean_data = utils.get_same_row_col_labels(raw_data)

In [14]:
get_organ_var_likelihood_debug(0, 2, clean_data, geom_normalize=True, use_common_var_2=True)

[5.87339568e+00 1.39368200e-32 4.60893773e-44 4.05361685e-05
 2.33089611e+00 2.49459691e+00 0.00000000e+00 4.60545475e-14
 1.45553620e+01]
0.0
== lh values== 
         bone-brain   bone-fat bone-kidney bone-liver  bone-lung bone-muscle  \
0h-0h     5.873396   13.47125    3.408615  10.218065  10.801185    8.445347   
0h-24h         0.0        0.0    0.019701   0.000017   0.000001         0.0   
0h-3h          NaN   12.84703    0.723203   2.116823        NaN    6.236252   
0h-48h         0.0   0.000001    0.008825   0.000001   0.000811         0.0   
0h-8h          NaN        NaN         NaN        NaN        NaN         NaN   
24h-0h    0.000041   2.580358    0.000584     0.9452   2.193016    0.008379   
24h-24h   2.330896   1.760228    4.332739   4.289708    2.76268    1.272801   
24h-3h         NaN   2.556511     1.71663   3.272827        NaN    0.015622   
24h-48h   2.494597   2.291912    6.127668    5.93783   3.664935    0.765538   
24h-8h         NaN        NaN         NaN        N

,bone,brain,fat,kidney,liver,lung,muscle
bone,1.38396,0.0,0.045603,0.00003,0.003472,0.240158,0.0
brain,NaN,1.669204,3.015062,0.337739,4.326858,2.86713,0.0
fat,NaN,NaN,0.389098,0.017424,0.002299,0.148977,0.13801
kidney,NaN,NaN,NaN,1.289755,0.712951,0.318837,0.0
liver,NaN,NaN,NaN,NaN,1.105597,0.000016,0.0
lung,NaN,NaN,NaN,NaN,NaN,0.021555,0.0
muscle,NaN,NaN,NaN,NaN,NaN,NaN,0.0


# Debugging

In [15]:
a = utils.get_var_pair_df(('bone', 'brain'), data=clean_data)

b = utils.get_var_pair_df(('0h', '0h'), data=a)
b

,brain_BL_0h_r1,brain_BL_0h_r2,brain_BL_0h_r3,brain_BL_0h_r4,brain_BL_0h_r5,brain_BL_0h_r6,brain_BL_0h_r7,brain_BL_0h_r8,brain_BL_0h_r9
bone_BL_0h_r6,0.634599,0.702376,0.648401,0.633350,0.659042,0.651780,0.646761,0.719912,0.720934
bone_BL_0h_r7,0.667836,0.643652,0.721478,0.704047,0.611087,0.649846,0.703551,0.706947,0.772451


In [16]:
utils.get_var_pair_df(('3h', '3h'), data=a)

,brain_BL_3h_r1
bone_BL_3h_r1,0.112416
